In [3]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [4]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA
import numpy as np

def knapsack_quadratic_program():
    # Put values, weights and max_weight parameter for the Knapsack()
    values = [10, 13, 18, 31, 7]
    weights = [2, 4, 6, 7, 3]
    max_weight = 10
    ##############################
    # Provide your code here
    
    
    prob = Knapsack(values, weights, max_weight)
    
    ##############################
    # to_quadratic_program generates a corresponding QuadraticProgram of the instance of the knapsack problem.
    kqp = prob.to_quadratic_program()
    return prob, kqp

prob,quadratic_program=knapsack_quadratic_program()
quadratic_program

<QuadraticProgram: maximize 10*x_0 + 13*x_1 + 18*x_2 + 31*x_3 + 7*x_4, 5 variables, 1 constraints, 'Knapsack'>

We can solve the problem using the classical NumPyMinimumEigensolver to find the minimum eigenvector, which may be useful as a reference without doing things by Dynamic Programming; we can also apply QAOA.

In [5]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 

result:
 fval=41.0, x_0=1.0, x_1=0.0, x_2=0.0, x_3=1.0, x_4=0.0, status=SUCCESS

 index of the chosen items: [0, 3]


In [14]:
# QAOA
from qiskit.primitives import Sampler
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.algorithms.optimizers import COBYLA

seed = 123
algorithm_globals.random_seed = seed
sampler = Sampler()
aer_sampler = AerSampler(run_options={"shots": 1000, "seed": seed})

optimizer = COBYLA()

qaoa = QAOA(aer_sampler, optimizer, reps=2)
meo = MinimumEigenOptimizer(min_eigen_solver= qaoa)
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 



result:
 fval=41.0, x_0=1.0, x_1=0.0, x_2=0.0, x_3=1.0, x_4=0.0, status=SUCCESS

 index of the chosen items: [0, 3]


O circuito do Quantum Approximate Optimization Algorithm (QAOA) é composto por uma série de camadas quânticas, onde cada camada consiste em uma aplicação iterativa de dois tipos de operadores: um operador de mistura e um operador de problema. O número de camadas é controlado por um parâmetro inteiro p.

Operador de Mistura (Mixer Operator): Este operador é projetado para criar superposições de estados. Ele é geralmente implementado como uma sequência de portas Hadamard aplicadas a cada qubit individualmente ou portas de X (bit-flip) entre pares de qubits, e é usado para explorar diferentes combinações de estados.

Operador de Problema (Cost Operator): Este operador codifica o problema de otimização que você deseja resolver em um formato quântico. Ele geralmente envolve mapear o problema em um Hamiltoniano quântico, que é uma soma ponderada de Pauli-Z operators em cada qubit, e as ponderações são os coeficientes do problema de otimização.

O circuito geral do QAOA é uma alternância de camadas quânticas de operadores de mistura e operadores de problema. Isso é feito p vezes (onde p é o parâmetro que controla o número de camadas). Além disso, antes e depois de todas as camadas, você pode aplicar portas de Hadamard para criar uma superposição inicial de estados e medir os qubits no final para obter a solução aproximada do problema de otimização.

A complexidade e o desempenho do QAOA dependem fortemente do número de camadas (p) e da escolha dos operadores de mistura e problema apropriados para o problema específico que você está resolvendo.

O circuito do QAOA é uma representação quântica de um problema de otimização, onde a busca pela solução ótima é realizada por meio de manipulações quânticas. O objetivo final é encontrar a configuração de qubits que minimize a função de custo associada ao problema de otimização. No entanto, a definição exata do circuito pode variar dependendo do problema que está sendo resolvido.







In [22]:
def knapsack_quadratic_program(values, weights, max_weight):
    # Crie uma instância de QuadraticProgram
    qp = QuadraticProgram()

    # Defina as variáveis binárias para representar a escolha de cada item
    num_items = len(values)
    x = qp.binary_var_list(name="x", length=num_items)

    # Defina a função objetivo para maximizar o valor total dos itens escolhidos
    qp.maximize(linear=[values[i] * x[i] for i in range(num_items)])

    # Adicione a restrição de que o peso total não pode exceder a capacidade máxima
    qp.linear_constraint(linear=[weights[i] * x[i] for i in range(num_items)], sense="LE", rhs=max_weight, name="weight_constraint")

    return qp

# Dados do problema da mochila
values = [10, 13, 18, 31, 7]
weights = [2, 4, 6, 7, 3]
max_weight = 10

# Criar o problema de otimização quadrática (QUBO) correspondente
qubo = knapsack_quadratic_program(values, weights, max_weight)

Traceback (most recent call last):
  Cell In[22], line 23
    qubo = knapsack_quadratic_program(values, weights, max_weight)
  Cell In[22], line 7 in knapsack_quadratic_program
    x = qp.binary_var_list(name="x", length=num_items)
TypeError: QuadraticProgram.binary_var_list() got an unexpected keyword argument 'length'

Use %tb to get the full traceback.
